In [1]:
import sys
sys.path.append("src")

import dataclasses
import numpy as np
import itertools
from matplotlib import pyplot as plt

import statfin
import verolysis

In [2]:
db = statfin.PxWebAPI.Verohallinto()
tbl = db.table("Vero", "tulot_101.px")

In [3]:
BITS = [
    "HVT_TULOT_10",  # Lukumäärä
    "HVT_TULOT_70",  # Ansiotulot
    "HVT_TULOT_80",  # Palkkatulot
    "HVT_TULOT_280", # Eläketulot
]

In [4]:
df = tbl.query({
    "Verovuosi": 2022,
    "Erä": BITS,
    "Tulonsaajaryhmä": "*",
    "Tuloluokka": "*",
    "Tunnusluvut": "*",
})

In [6]:
def get_cell_row(df, grp, cls, bit):
    """Poimi soluindeksin ja erän mukainen rivi"""
    df = df[(df.Tulonsaajaryhmä == str(grp)) & (df.Tuloluokka == str(cls)) & (df.Erä == bit)]
    assert len(df) == 1, len(df)
    return df.iloc[0]

In [7]:
def get_cell_size(df, grp, cls):
    """Henkilöiden lukumäärä solussa"""
    return get_cell_row(df, grp, cls, "HVT_TULOT_10").N

In [26]:
def iter_cell_keys_gc(df):
    """
    Soluindeksit tulonsaajaryhmän ja tuloluokan mukaan

    Soluindeksi on pari (tulonsaajaryhmä, tuloluokka). Tämä palauttaa
    listan niistä indekseistä, jotka dataframesta löytyvät.
    """
    df = df[(df.Tulonsaajaryhmä != "Y") & (df.Tuloluokka != "SS")]
    i = 0
    for g in sorted(df.Tulonsaajaryhmä.unique().astype(int)):
        for c in sorted(df.Tulonsaajaryhmä.unique().astype(int)):
            N = get_cell_size(df, g, c)
            if np.isfinite(N) and N > 0:
                N = int(N)
                k = int(1 + (N // 10))
                yield g, c, N, k, i
                i += k

In [8]:
def get_cell_density(df, grp, cls, bit):
    """Solun erän tiheysfunktio"""
    row = get_cell_row(df, grp, cls, bit)
    if row.Mean is None:
        return None
    return verolysis.income_brackets.row_to_density(row)

In [55]:
def sample_cell(df, grp, cls, bit, k):
    """Poimi otanta solusta"""
    N = get_cell_size(df, grp, cls)
    d = get_cell_density(df, grp, cls, bit)
    if d is None:
        return np.zeros(k)
    s = d.uniform_sample(k, leftpad=N, left=0)
    if any(np.isnan(s)):
        return np.zeros(k)
    return s

In [63]:
keys = list(iter_cell_keys_gc(df))
M = np.sum([N for _, _, N, _, _ in keys])
N = np.sum([k for _, _, _, k, _ in keys])
print(f"{int(N):,} samples of {int(M):,} persons in {len(keys)} cells")

bits = ["HVT_TULOT_70", "HVT_TULOT_80", "HVT_TULOT_280"]
V = {}
for bit in bits:
    v = np.zeros(N)
    for g, c, _, k, i in keys:
        s = sample_cell(df, g, c, bit, k)
        v[i:i+k] = s
    V[bit] = v

466,253 samples of 4,661,862 persons in 132 cells


In [86]:
def rnd():
    import random
    ki = random.randint(0, len(keys)-1)
    g, c, _, k, i = keys[ki]
    ii = random.randint(0, k-1)
    d = tbl.values["Tulonsaajaryhmä"]
    G = d[d.code == str(g)].iloc[0].text
    print(f"Tulonsaajaryhmä: {G} ({g})")
    d = tbl.values["Tuloluokka"]
    C = d[d.code == str(c)].iloc[0].text
    print(f"Tuloluokka:      {C} ({c})")
    for bit in bits:
        v = V[bit]
        print(f"{bit+':':<16} {v[i+ii]:.0f}")

for i in range(5):
    rnd()

Tulonsaajaryhmä: Pääomatulon saaja (11)
Tuloluokka:      45 000 – 49 999 (10)
HVT_TULOT_70:    19587
HVT_TULOT_80:    0
HVT_TULOT_280:   11920
Tulonsaajaryhmä: Pääomatulon saaja (11)
Tuloluokka:      45 000 – 49 999 (10)
HVT_TULOT_70:    11706
HVT_TULOT_80:    0
HVT_TULOT_280:   0
Tulonsaajaryhmä: Muut (7)
Tuloluokka:      60 000 – 64 999 (13)
HVT_TULOT_70:    63069
HVT_TULOT_80:    0
HVT_TULOT_280:   0
Tulonsaajaryhmä: Elinkeinonharjoittaja (4)
Tuloluokka:      15 000 – 19 999 (4)
HVT_TULOT_70:    19383
HVT_TULOT_80:    3342
HVT_TULOT_280:   0
Tulonsaajaryhmä: Maatalouden harjoittaja (3)
Tuloluokka:      45 000 – 49 999 (10)
HVT_TULOT_70:    23572
HVT_TULOT_80:    0
HVT_TULOT_280:   0


In [84]:
def between(x, a, b=1e12):
    return np.clip(x, a, b) - a
    

@dataclasses.dataclass
class Ansiotulovähennys:
    cap: float = 3_570
    r1: float = 2_500
    r2: float = 7_230
    r3: float = 14_000
    t1: float = (51 / 100)
    t2: float = (28 / 100)
    t3: float = (4.5 / 100)
    
    def __call__(self, palkkatulot, puhtaat_ansiotulot):
        up1 = self.t1 * between(palkkatulot, self.r1, self.r2)
        up2 = self.t2 * between(palkkatulot, self.r2)
        up = np.maximum(up1 + up2, self.cap)
        dn = self.t3 * between(puhtaat_ansiotulot, self.r3)
        return np.maximum(up - dn, 0)


@dataclasses.dataclass
class Perusvähennys:
    cap: float = 3_740
    rate: float = (18 / 100)

    def __call__(self, tulot):
        up = np.minimum(tulot, self.cap)
        dn = self.rate * between(tulot, self.cap)
        return np.maximum(up - dn, 0)

class Eläketulovähennys

SyntaxError: expected ':' (3544687557.py, line 33)

In [ ]:
ATV = Ansiotulovähennys()
PV = Perusvähennys()

print(ATV(10_000, 15_000))
print(PV(4_000))

In [ ]:
r_tyel = (7.15 / 100)
r_tvm = (1.50 / 100)

pt = 10_000
et = 10_000
vv = 1_000

tyel = r_tyel * pt
tvm = r_tvm * pt
at1 = pt + et - tyel - tvm
pat = at0 - min(pat0, 750)
pat1 = pat vv
pat2 -= ATV(pt, pat)
pat3 -= PV(pat2)
vt = 